In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tabela = pd.read_csv('clientes.csv')
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

None

Preparar a Base de Dados Para a Inteligência Artificial

In [14]:
#tratar colunas de texto: profissao, mix_credito e comportamento_pagamento

codificador = LabelEncoder() 
tabela['profissao'] = codificador.fit_transform(tabela['profissao']) # codifica a coluna transformando os textos em numeros
tabela['mix_credito'] = codificador.fit_transform(tabela['mix_credito'])
tabela['comportamento_pagamento'] = codificador.fit_transform(tabela['comportamento_pagamento'])
display(tabela.info())

# agora ao inves de object, as colunas são classificadas como int


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  int32  
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  int32  
 13  divida_total              100000 non-null  fl

None

In [15]:
x = tabela.drop(columns=['score_credito', 'id_cliente']) #quem a ia usa para fazer a previsão
y = tabela['score_credito'] #quem a ia quer prever

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)


Criar o Modelo de IA -> Nota de Crédito: Ruim, Média ou Boa

In [16]:
# Random Florest -> Arvore de Decisão
# Neirest Neighbors -> KNN -> Vizinhos Próximos

# - Importar IA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# - Criar IA
model_randomflorest = RandomForestClassifier()
model_knn = KNeighborsClassifier()

# - Treinar IA
model_randomflorest.fit(x_train, y_train) #Perguntas pra base de dados 
model_knn.fit(x_train, y_train) # Separa a base de dados de acordo com os vizinhos próximos

KNeighborsClassifier()

Escolher o Modelo

In [17]:
previsao_randomflorest = model_randomflorest.predict(x_test)
previsao_knn = model_knn.predict(x_test)

# calcular acurácia

from sklearn.metrics import accuracy_score

display(accuracy_score(y_test, previsao_randomflorest))
display(accuracy_score(y_test, previsao_knn))

# modelo arvore de decisao teve mais acurácia, ela será escolhida

0.8282666666666667

0.7396666666666667

Usar Modelo Para Fazer Novas Previsões

In [18]:
# importar nova tabela
novos_clientes = pd.read_csv('novos_clientes.csv')

# tratar colunas
novos_clientes['profissao'] = codificador.fit_transform(novos_clientes['profissao']) 
novos_clientes['mix_credito'] = codificador.fit_transform(novos_clientes['mix_credito'])
novos_clientes['comportamento_pagamento'] = codificador.fit_transform(novos_clientes['comportamento_pagamento'])

prev_novos_clientes = model_randomflorest.predict(novos_clientes)
display(novos_clientes)
display(prev_novos_clientes)



,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,31.0,1,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,29.934186,218.0,44.50951,1,312.487689,1,1,0,0,0
1,4,32.0,0,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,28.819407,12.0,0.00000,2,300.994163,0,0,0,0,1
2,2,48.0,1,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,34.235853,215.0,0.00000,0,345.081577,0,1,0,1,0


array(['Poor', 'Standard', 'Standard'], dtype=object)